# Introduction
This project will be about making a license plate detection model using Yolov11. 

## The project goals are:
1. Upload a car image from the front or back (where the license plate is).
2. Segment/Extract the license plate from the image.
3. Recognize and extract the text written using OCR.

# 1. Importing Libraries

In [6]:
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import random
from PIL import Image

# 2. Importing the Dataset
Count the number of files in the dataset

In [2]:
# Function to count the images in a directory
def get_directory(directory, structure=['images', 'labels']):
    folders = os.listdir(directory)
    for folder in folders:
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            subfolders = os.listdir(folder_path)
            for final_folders in subfolders:
                if final_folders in structure:
                    final_folder_path = os.path.join(folder_path, final_folders)
                    files = [f for f in os.listdir(final_folder_path) if os.path.isfile(os.path.join(final_folder_path, f))]
                    num_files = len(files)
                    print(f"The Number of files in {final_folder_path} is {num_files}.")

# Print the number of files in each directory
get_directory("data")

The Number of files in data\test\images is 1020.
The Number of files in data\test\labels is 1020.
The Number of files in data\train\images is 7057.
The Number of files in data\train\labels is 7057.
The Number of files in data\valid\images is 2048.
The Number of files in data\valid\labels is 2048.


# 3. Training the Model
This section will be about training the model to segment the license plate from the image.

In [3]:
# Assigning the model
model = YOLO("yolo11s.pt")

# Train settings
epochs = 25
input_image_size = 640
patience = 3
batch = 32
val = True
cache = False
plots = True
project = "Yolov11-License-Plate-Model"
workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Clear GPU Cache
torch.cuda.empty_cache()

# Train the model
results = model.train(
    data="data/data.yaml",
    epochs=epochs,
    imgsz=input_image_size,
    patience=patience,
    batch=batch,
    val=val,
    cache=cache,
    plots=plots,
    project=project,
    workers=workers,
    device=device,
)

New https://pypi.org/project/ultralytics/8.3.229 available  Update with 'pip install -U ultralytics'
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=3, perspective=0.0, plots=True, pos

# 4. Inference

In [5]:
# Load the best trained model
model = YOLO(r"I:\AI-ML-Projects\Yolov11-License Plate Recognition\Yolov11-License-Plate-Model\train5\weights\best.pt")

In [11]:
def read_random_images(folder_path, num_samples=10):
    """
    Read random images from a folder
    
    Args:
        folder_path (str): Path to the folder containing images
        num_samples (int): Number of images to read
    
    Returns:
        list: List of PIL Image objects
    """
    # Get all image files from the folder
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
    image_files = [
        f for f in os.listdir(folder_path) 
        if os.path.splitext(f)[1].lower() in image_extensions
    ]
    
    # Check if we have enough images
    if len(image_files) < num_samples:
        raise ValueError(f"Only {len(image_files)} images found, but {num_samples} requested")
    
    # Randomly select images
    selected_files = random.sample(image_files, num_samples)
    
    # Read and return images
    images = []
    for filename in selected_files:
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        images.append(img)
    
    return images


# Load the test images
test_dir_path = "data/test/images"
test_images = read_random_images(test_dir_path, 10)

# Perform inference on the test images
def perform_inference(model, images_path, output_dir, batch_size=8):
    """
    Function to perform inference on a list of images.

    Args:
        model (YoloModel): The YOLO model to use for inference.
        images_path (list): A list of image file paths.
        output_dir (str): The directory to save the output images.
        batch_size (int): The batch size for inference.

    Returns:
        The results of the inference.
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Perform inference
    results = model(
        images_path,
        save=True,
        save_txt=True,
        save_conf=True,
        project=output_dir,
        batch=batch_size
    )

    return results

# Perform inference on the test images
output_inf_dir = "inference results"
results = perform_inference(model, test_images, output_inf_dir, 8)
print(f"Results saved in directory {output_inf_dir} for {len(results)} images")


0: 640x640 1 License_Plate, 4.6ms
1: 640x640 1 License_Plate, 4.6ms
2: 640x640 1 License_Plate, 4.6ms
3: 640x640 1 License_Plate, 4.6ms
4: 640x640 1 License_Plate, 4.6ms
5: 640x640 1 License_Plate, 4.6ms
6: 640x640 1 License_Plate, 4.6ms
7: 640x640 1 License_Plate, 4.6ms
8: 640x640 1 License_Plate, 4.6ms
9: 640x640 1 License_Plate, 4.6ms
Speed: 1.7ms preprocess, 4.6ms inference, 1.9ms postprocess per image at shape (8, 3, 640, 640)
Results saved to I:\AI-ML-Projects\Yolov11-License Plate Recognition\inference results\predict3
10 labels saved to I:\AI-ML-Projects\Yolov11-License Plate Recognition\inference results\predict3\labels
Results saved in directory inference results for 10 images


# 5. Extracting text from the license plates
In this section, we will extract text from the license plate using python library easyocr, but first we have to preprocess the image to crop out the license plates detected from the image. So the project flow will be like:

1. Model detected the license plate(s) from the image.
2. Crop the license plate(s) from the image.
3. Extract the text from the license plate(s).

In [ ]:
# Function to preprocess the images
def crop_license_plate(image):
    # Read the image
    img = cv2.imread(image)
    